In [1]:
def download(item):
    ...

def resize(item):
    ...

def upload(item):
    ...

In [2]:
from collections import deque
from threading import Lock

class MyQueue:
	def __init__(self):
		self.items = deque()
		self.lock = Lock()
	
	def put(self, item):
		with self.lock:
			self.items.append(item)
	
	def get(self, item):
		with self.lock:
			self.items.popleft(item)

In [3]:
def put(self, item):
    with self.lock:
        self.items.apend(item)

In [4]:
def get(self, item):
    with self.lock:
        self.items.popleft(item)

In [5]:
from threading import Thread
import time

class Worker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super().__init__()
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.polled_count = 0
        self.work_done = 0

In [6]:
def run(self):
    while True:
        self.polled_count += 1
        try :
            item = self.in_queue.get()
        except IndexError:
            time.sleep(0.01) # 할 일이 없음
        else :
            result = self.fun(item)
            self.out_queue.put(result)
            self.work_done += 1

In [7]:
download_queue = MyQueue()
resize_queue = MyQueue()
upload_queue = MyQueue()
done_queue = MyQueue()

threads = [
    Worker(download, download_queue, resize_queue),
    Worker(resize, resize_queue, upload_queue),
    Worker(upload, upload_queue, done_queue)
]

In [8]:
for thread in threads:
    thread.start()

for _ in range(1000):
    download_queue.put(object())

In [51]:
while len(done_queue.items) < 1000:
    # 기다리는 도안 유용한 작업을 수행한다.
    time.sleep(0.1)
    
for thread in threads:
    thread.in_queue = None
    thread.join()

KeyboardInterrupt: 

In [9]:
processed = len(done_queue.items)
polled = sum(t.polled_count for t in threads)
print(f'{processed} 개의 아이템을 처리했습니다,'
      f'이때 풀링을 {polled}번 했습니다.')

0 개의 아이템을 처리했습니다,이때 풀링을 0번 했습니다.


In [10]:
from queue import Queue

my_queue = Queue()

def consumer():
    print('소비자 대기')
    my_queue.get() # 다음에 보여줄 put()이 실행된 다음에 실행된다.
    print('소비자 완료')

thread = Thread(target=consumer)
thread.start()

소비자 대기


In [11]:
print('생산자 데이터 추가')
my_queue.put(object()) # 앞에서 본 get()이 실행되기 전에 실행된다.
print('생산자 완료')
thread.join()

생산자 데이터 추가
생산자 완료
소비자 완료


In [12]:
my_queue = Queue(1) # 버퍼크기 1

def consumer():
    time.sleep(0.1) # 대기
    my_queue.get() # 두번째로 실행됨
    print('소비자 1')
    my_queue.get() # 네번째로 실행됨
    print('소비자 2')
    print('소비자 완료')

thread = Thread(target=consumer)
thread.start()

In [13]:
# Example 14
my_queue.put(object())          # Runs first
print('생산자 1')
my_queue.put(object())          # Runs third
print('생산자 2')
print('생산자 완료')
thread.join()

생산자 1소비자 1

생산자 2
생산자 완료
소비자 2
소비자 완료


In [14]:
class ClosableQueue(Queue):
    SENTINEL = object()

    def close(self):
        self.put(self.SENTINEL)

    def __iter__(self):
        while True:
            item = self.get()
            try:
                if item is self.SENTINEL:
                    return # 스레드를 종료시킨다.
                yield item
            finally:
                self.task_done()

In [15]:
class StoppableWorker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super().__init__()
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
    
    def run(self):
        for item in self.in_queue:
            result = self.func(item)
            self.out_queue.put(result)

In [16]:
download_queue = ClosableQueue()
resize_queue = ClosableQueue()
upload_queue = ClosableQueue()
done_queue = ClosableQueue()

threads = [
    StoppableWorker(download, download_queue, resize_queue),
    StoppableWorker(resize, resize_queue, upload_queue),
    StoppableWorker(upload, upload_queue, done_queue)
]

In [23]:
for thread in threads :
    thread.start()

for _ in range(1000):
    download_queue.put(object())

done_queue.close()

In [24]:
download_queue.join()
resize_queue.close()
resize_queue.join()
upload_queue.close()
upload_queue.join()
print(done_queue.qsize(), '개의 원소가 처리됨')

for thread in threads:
    thread.join()

1001 개의 원소가 처리됨


In [17]:
def start_threads(count, *args):
    threads = [StoppableWorker(*args) for _ in range(count)]
    for thread in threads:
        thread.start()
    return threads

def stop_threads(closable_queue, threads):
    for _ in threads:
        closable_queue.close()

    closable_queue.join()

    for thread in threads:
        thread.join()


In [18]:
download_queue = ClosableQueue()
resize_queue = ClosableQueue()
upload_queue = ClosableQueue()
done_queue = ClosableQueue()

download_threads = start_threads(
    3, download, download_queue, resize_queue)
resize_threads = start_threads(
    4, resize, resize_queue, upload_queue)
upload_threads = start_threads(
    5, upload, upload_queue, done_queue)

for _ in range(1000):
    download_queue.put(object())

stop_threads(download_queue, download_threads)
stop_threads(resize_queue, resize_threads)
stop_threads(upload_queue, upload_threads)

print(done_queue.qsize(), '개의 원소가 처리됨')

1000 개의 원소가 처리됨
